# Neural Network Model


In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd

# # Import and read the csv.
# b2 view
df = pd.read_csv("../ML_Data_&_Preprocessing/b2_df_nonEncoded.csv", index_col=[0])
df.head()



,state,spend,Impressions,leads,link_clicks,reach,Agency Tiers
0,IL,47.33,1780,0,7,1689,4.0
1,NY,63.65,1857,1,10,1737,4.0
2,OK,32.53,1718,0,7,1527,4.0
3,SC,32.31,1725,1,14,1645,4.0
4,CA,101.13,3745,1,28,3513,4.0


In [2]:
df['leads'].value_counts()

1    21698
0    12272
Name: leads, dtype: int64

In [3]:
# check on different state values for testing 
df['state'].value_counts()

CA    7685
TX    4254
VA    3239
CO    2386
FL    1758
MI    1655
GA    1620
NY    1486
NC    1050
TN     970
WA     960
NJ     919
CT     893
OH     842
MO     777
MN     628
IL     605
MA     454
SC     365
OK     365
MD     357
DC     250
SD     246
NE     173
PA      33
Name: state, dtype: int64

In [4]:
df2 = df.loc[df['state']== 'CA']
df2

,state,spend,Impressions,leads,link_clicks,reach,Agency Tiers
4,CA,101.13,3745,1,28,3513,4.0
5,CA,31.40,1764,1,30,1641,4.0
10,CA,40.95,2263,1,16,2107,4.0
17,CA,48.71,2400,1,21,2152,4.0
19,CA,18.52,705,1,14,672,4.0
...,...,...,...,...,...,...,...
33533,CA,77.28,1623,0,16,1455,NaN
33534,CA,11.49,404,0,5,398,NaN
33535,CA,18.03,576,0,5,559,NaN
33536,CA,17.27,555,0,7,537,NaN


In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7685 entries, 4 to 33537
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state         7685 non-null   object 
 1   spend         7685 non-null   float64
 2   Impressions   7685 non-null   int64  
 3   leads         7685 non-null   int64  
 4   link_clicks   7685 non-null   int64  
 5   reach         7685 non-null   int64  
 6   Agency Tiers  3175 non-null   float64
dtypes: float64(2), int64(4), object(1)
memory usage: 480.3+ KB


In [6]:
df3 = df2.drop(df2.columns[[0,6]], axis=1)

In [7]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7685 entries, 4 to 33537
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   spend        7685 non-null   float64
 1   Impressions  7685 non-null   int64  
 2   leads        7685 non-null   int64  
 3   link_clicks  7685 non-null   int64  
 4   reach        7685 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 360.2 KB


# Split into train and test features

In [8]:
# Split our preprocessed data into our features and target arrays
y = df3['leads'].values
X = df3.drop(['leads'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\rashi\AppData\Local\Temp\ipykernel_15460\3165784757.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df3.drop(['leads'], 1).values


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train, Evaluate our Model 1 - NN

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# using multi-layer perceptron (two layers)
numInputFeatures = len(X_train[0])

# I have X amount of columns
# layer1 = input layer, typically equals number of input variables in data
layer1 = 30
# layer 2 = hidden layer, typically 2/3 of input layer
layer2 = 15
# layer 3 = hidden layer
layer3= 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer1, 
                          input_dim=numInputFeatures, 
                          activation="hard_sigmoid")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, 
                             activation="elu"))

# adding a third layer to increase accuracy 
nn.add(tf.keras.layers.Dense(units=layer3, 
                             activation='elu'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="hard_sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                150       
                                                                 
 dense_1 (Dense)             (None, 15)                465       
                                                                 
 dense_2 (Dense)             (None, 2)                 32        
                                                                 
 dense_3 (Dense)             (None, 1)                 3         
                                                                 
Total params: 650
Trainable params: 650
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    # checkpoint directory and file structure defined above
    filepath=checkpoint_path,
    # notified when checkpoint is being saved to the directory
    verbose=1,
    # checkpoint files take small space
    save_weights_only=True,
    # checkpoints saved every epoch
    save_freq='epoch')

In [12]:
# Train the model
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
164/181 [==========================>...] - ETA: 0s - loss: 0.5695 - accuracy: 0.7081
Epoch 1: saving model to checkpoints\weights.01.hdf5
181/181 [==============================] - 1s 2ms/step - loss: 0.5673 - accuracy: 0.7080
Epoch 2/100
176/181 [============================>.] - ETA: 0s - loss: 0.5194 - accuracy: 0.7811
Epoch 2: saving model to checkpoints\weights.02.hdf5
181/181 [==============================] - 0s 2ms/step - loss: 0.5200 - accuracy: 0.7803
Epoch 3/100
177/181 [============================>.] - ETA: 0s - loss: 0.5074 - accuracy: 0.7848
Epoch 3: saving model to checkpoints\weights.03.hdf5
181/181 [==============================] - 0s 2ms/step - loss: 0.5063 - accuracy: 0.7854
Epoch 4/100
170/181 [===========================>..] - ETA: 0s - loss: 0.4985 - accuracy: 0.7790
Epoch 4: saving model to checkpoints\weights.04.hdf5
181/181 [==============================] - 0s 3ms/step - loss: 0.4991 - accuracy: 0.7786
Epoch 5/100
154/181 [=======================

160/181 [=========================>....] - ETA: 0s - loss: 0.4481 - accuracy: 0.8014
Epoch 69: saving model to checkpoints\weights.69.hdf5
181/181 [==============================] - 0s 2ms/step - loss: 0.4465 - accuracy: 0.8015
Epoch 70/100
170/181 [===========================>..] - ETA: 0s - loss: 0.4433 - accuracy: 0.8024
Epoch 70: saving model to checkpoints\weights.70.hdf5
181/181 [==============================] - 0s 3ms/step - loss: 0.4419 - accuracy: 0.8032
Epoch 71/100
168/181 [==========================>...] - ETA: 0s - loss: 0.4420 - accuracy: 0.7987
Epoch 71: saving model to checkpoints\weights.71.hdf5
181/181 [==============================] - 1s 3ms/step - loss: 0.4418 - accuracy: 0.7996
Epoch 72/100
179/181 [============================>.] - ETA: 0s - loss: 0.4427 - accuracy: 0.8012
Epoch 72: saving model to checkpoints\weights.72.hdf5
181/181 [==============================] - 0s 2ms/step - loss: 0.4428 - accuracy: 0.8015
Epoch 73/100
163/181 [==========================>

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# when brand 1, all states, accuracy = 36%
# when brand 2, all states, accuracy = 36%
# when brand 3, all states, accuracy = 36%

# CA = .80%
# FL = .84%
# TX = .84%
# VA = .78%
# CO = .71%
# MI = .79%
# GA = .84%
# NY = .85%
# NC = .85%
# TN = .75%
# WA = .80%
# NJ = .78%
# CT = .82%
# OH = .82%
# MO = .91%
# MN = .77%
# IL = .82%
# MA = .85%
# SC = .63%
# OK = .72%
# MD = .89%
# DC = .84%
# SD = .94%
# NE = .86%
# PA = 100%


61/61 - 1s - loss: 0.4750 - accuracy: 0.7789 - 533ms/epoch - 9ms/step
Loss: 0.47504448890686035, Accuracy: 0.7788761854171753


### B2 Results

#### B2

In [14]:
data_results_b2 = {
        'State': ['CA', 'FL', 'TX', 'VA', 'CO', 'MI', 'GA', 'NY', 'NC', 'TN', 'WA', 'NJ', 'CT', 'OH', 'MO', 'MN', 'IL', 'MA', 'SC', 'OK', 'MD', 'DC', 'DC', 'SD', 'NE', 'PA'],
        'Predictive Accuracy': [.80, .84, .84, .78, .71, .79, .84, .85, .85, .75  ],
        'Notes': ['N/A', 'N/A', 'N/A', 'N/A','N/A','N/A','N/A','N/A','N/A','N/A' ]}
data_results_b2

{'State': ['CA',
  'FL',
  'TX',
  'VA',
  'CO',
  'MI',
  'GA',
  'NY',
  'NC',
  'TN',
  'WA',
  'NJ',
  'CT',
  'OH',
  'MO',
  'MN',
  'IL',
  'MA',
  'SC',
  'OK',
  'MD',
  'DC',
  'DC',
  'SD',
  'NE',
  'PA'],
 'Predictive Accuracy': [0.8,
  0.84,
  0.84,
  0.78,
  0.71,
  0.79,
  0.84,
  0.85,
  0.85,
  0.75],
 'Notes': ['N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A']}

In [15]:
# create the new df to display brand, state, and model accuracy 
df_results_b2 = pd.DataFrame(data_results_b2)
df_results_b2

ValueError: All arrays must be of the same length

In [ ]:
# # Export the model to HDF5 file
# nn.save("AlphabetSoupCharity_optimization.h5")